# Parameter Management

The ultimate goal of training deep networks is to find good parameter values for a given architecture. When everything is standard, the __torch.nn.Sequential__ class is a perfectly good tool for it. However, very few models are entirely standard and most scientists want to build things that are novel. This section shows how to manipulate parameters. In particular we will cover the following aspects:

- Accessing parameters for debugging, diagnostics, to visualize them or to save them is the first step to understanding how to work with custom models.
- Secondly, we want to set them in specific ways, e.g. for initialization purposes. We discuss the structure of parameter initializers.
- Lastly, we show how this knowledge can be put to good use by building networks that share some parameters.

As always, we start from our trusty Multilayer Perceptron with a hidden layer. This will serve as our choice for demonstrating the various features.

In [ ]:
import torch
import torch.nn as nn

net = nn.Sequential()
net.add_module('Linear_1', nn.Linear(20, 256, bias = False))
net.add_module('relu', nn.ReLU())
net.add_module('Linear_2', nn.Linear(256, 10, bias = False))

# the init_weights function initializes the weights of our multi-layer perceptron 

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

# the net.apply() applies the above stated initialization of weights to our net        

net.apply(init_weights) 

x = torch.randn(2,20)   #initialing a random tensor of shape (2,20)
net(x)  #Forward computation


## Parameter Access


In the case of a Sequential class we can access the parameters with ease, simply by calling __net.parameters__. Let’s try this out in practice by inspecting the parameters.

In [ ]:
print(net[0].parameters)
print(net[1].parameters)
print(net[2].parameters)

The output tells us a number of things. Firstly, there are 3 layers; 2 linear layers and 1 ReLU layer as we would expect. The output also specifies the shapes that we would expect from linear layers. In particular the names of the parameters are very useful since they allow us to identify parameters uniquely even in a network of hundreds of layers and with nontrivial structure. Also, the output tells us that bias is __False__ as we specified it.  


### Targeted Parameters

In order to do something useful with the parameters we need to access them, though. There are several ways to do this, ranging from simple to general. Let’s look at some of them.

In [ ]:
print(net[0].bias)

It returns the bias of the first linear layer. Since we initialized the bias to be __False__, the output is None. We can also access the parameters by name, such as **Linear_1**. Both methods are entirely equivalent but the first method leads to much more readable code.

In [ ]:
print(net.Linear_1.weight)
print(net[0].weight)

Note that the weights are nonzero. This is by design since we applied __Xavier initialization__ to our network. We can also compute the gradient with respect to the parameters. It has the same shape as the weight. However, since we did not invoke backpropagation yet, the output is None.

In [ ]:
print(net[0].weight.grad)

### All parameters at once

Accessing parameters as described above can be a bit tedious, in particular if we have more complex blocks, or blocks of blocks (or even blocks of blocks of blocks), since we need to walk through the entire tree in reverse order to how the blocks were constructed. To avoid this, blocks come with a method __state_dict__ which grabs all parameters of a network in one dictionary such that we can traverse it with ease. It does so by iterating over all constituents of a block and calls __state_dict__ on subblocks as needed. To see the difference consider the following:

In [ ]:
print(net[0].state_dict) # parameters only for first layer
print(net.state_dict) # parameters for entire network

This provides us with a third way of accessing the parameters of the network. If we wanted to get the value of the weight term of the second linear layer we could simply use this:

In [ ]:
net.state_dict()['Linear_1.weight']

### Rube Goldberg strikes again

Let's see how the parameter naming conventions work if we nest multiple blocks inside each other. For that we first define a function that produces blocks (a block factory, so to speak) and then we combine these inside yet larger blocks.

In [ ]:
def block1():
    net = nn.Sequential(nn.Linear(16, 32),
                        nn.ReLU(),
                        nn.Linear(32, 16),
                        nn.ReLU())
    return net

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module('block'+str(i), block1())
    return net    
        
rgnet = nn.Sequential()
rgnet.add_module('model',block2())
rgnet.add_module('Last_linear_layer', nn.Linear(16,10))
rgnet.apply(init_weights)
x = torch.randn(2,16)
rgnet(x) # forward computation
    

Now that we are done designing the network, let's see how it is organized. __state_dict__ provides us with this information, both in terms of naming and in terms of logical structure.

In [ ]:
print(rgnet.parameters)
for param in rgnet.parameters():
    print(param.size(), param.dtype) 

Since the layers are hierarchically generated, we can also access them accordingly. For instance, to access the first major block, within it the second subblock and then within it, in turn the bias of the first layer, we perform the following.

In [ ]:
rgnet[0][1][0].bias.data

## Parameter Initialization

Now that we know how to access the parameters, let's look at how to initialize them properly. We discussed the need for initialization in section Numerical Stability. We often need to use methods to initialize the weights. PyTorch's init module provides a variety of preset initialization methods, but if we want something out of the ordinary, we need a bit of extra work. To initialize the weights of a single layer, we use a function from __torch.nn.init__ . For instance:

In [ ]:
linear1 = nn.Linear(2,5,bias=True)
torch.nn.init.normal_(linear1.weight, mean=0, std =0.01)  

If we wanted to initialize all parameters to 1, we could do this simply by changing the initializer to `Constant(1)`

In [ ]:
def init_weight(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight)
        
net = nn.Sequential()
net.add_module('Linear_1', nn.Linear(2, 5, bias = False))
net.add_module('Linear_2', nn.Linear(5, 5, bias = False))

net.apply(init_weight)
print(net.state_dict())

### Built-in Initialization

Let’s begin with the built-in initializers. The code below initializes all parameters with Gaussian random variables.

In [ ]:
def gaussian_normal(m):
    if type(m) == nn.Linear:
        torch.nn.init.normal_(m.weight)
        
net.apply(gaussian_normal)
print(net[0].weight)

If we wanted to initialize all parameters to 1, we could do this simply by changing the initializer to __torch.nn.init.constant_(tensor,1)__.

In [ ]:
def ones(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 1)
        
net.apply(ones)
print(net.state_dict())

If we want to initialize only a specific parameter in a different manner, we can simply set the initializer only for the appropriate subblock (or parameter) for that matter. For instance, below we initialize the __second layer__ to a constant value of __42__ and we use the __Xavier initializer__ for the weights of the __first layer__.

In [ ]:
block1 = nn.Sequential()
block1.add_module('Linear_1', nn.Linear(2,5,bias=False))
block2 = nn.Sequential()
block2.add_module('Linear_2', nn.Linear(5,5,bias=False))

model = nn.Sequential()
model.add_module('first', block1)
model.add_module('second', block2)

def xavier_normal(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)
              
block1.apply(xavier_normal)
block2.apply(init_42)
print(model.state_dict())

### Custom Initialization

Sometimes, the initialization methods we need are not provided in the init module. At this point, we can implement our desired implementation by writing the desired functions and use them to initialize the weights. In the example below, we pick a decidedly bizarre and nontrivial distribution, just to prove the point. We draw the coefficients from the following distribution:

$$ \begin{aligned} w \sim \begin{cases} U[5, 10] & \text{ with probability } \frac{1}{4} \   
0 & \text{ with probability } \frac{1}{2} \      
U[-10, -5] & \text{ with probability } \frac{1}{4} \end{cases} \end{aligned} $$

In [ ]:
def custom(m):
    torch.nn.init.uniform_(m[0].weight, -10,10)
    for i in range(m[0].weight.data.shape[0]):
        for j in range(m[0].weight.data.shape[1]):
            if m[0].weight.data[i][j]<=5 and m[0].weight.data[i][j]>=-5:
                m[0].weight.data[i][j]=0
    
    
m = nn.Sequential(nn.Linear(5,5,bias=False))
custom(m)
print(m.state_dict())

If even this functionality is insufficient, we can set parameters directly. Since __.data__ returns a Tensor we can access it just like any other matrix.

In [ ]:
m[0].weight.data +=1
m[0].weight.data[0][0] = 42
m[0].weight.data

## Tied Parameters

In some cases, we want to share model parameters across multiple layers. For instance when we want to find good word embeddings we may decide to use the same parameters both for encoding and decoding of words. Let’s see how to do this a bit more elegantly. In the following we allocate a linear layer and then use it multiple times for sharing the weights.

In [ ]:
# We need to give the shared layer a name such that we can reference its
# parameters

shared = nn.Sequential()
shared.add_module('linear_shared', nn.Linear(8,8,bias=False))
shared.add_module('relu_shared', nn.ReLU())                  
net = nn.Sequential(nn.Linear(20,8,bias=False),
               nn.ReLU(),
               shared,
               shared,
               nn.Linear(8,10,bias=False))

net.apply(init_weights)

print(net[2][0].weight==net[3][0].weight)


The above example shows that the parameters of the second and third layer are tied. They are identical rather than just being equal. That is, by changing one of the parameters the other one changes, too. 

## Summary

* We have several ways to access, initialize, and tie model parameters.
* We can use custom initialization.
* PyTorch has a sophisticated mechanism for accessing parameters in a unique and hierarchical manner.

## Exercises

1. Use the FancyMLP defined in :numref:`chapter_model_construction` and access the parameters of the various layers.
1. Look at the [PyTorch documentation](https://pytorch.org/docs/stable/_modules/torch/nn/init.html) and explore different initializers.
1. Try accessing the model parameters after `net.apply(initialization)` and before `net(x)` to observe the shape of the model parameters. What changes? Why?
1. Construct a multilayer perceptron containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?
